## How to calculate the Renewable Profiles like they do in PyPSA EUR. 

From PyPSA EUR contributors/authors.


weather cutouts can be downloaded from this zenodo repository: https://zenodo.org/records/12791128 

These files are merged spatiotemporal subsets of the European weather data from the ECMWF ERA5 realanlysis dataset and CMSAF SARAH-3 solar radiation dataset compiled by the atlite tool by various authors of PyPSA Eur 

The following script is modified from the build_renewable_profiles.py file in scripts of PyPSA-EUR. Running just soley this element of PYSPA-EUR caused issues with time alignment, thus I've replicated it here. This can also be used for any further edits of the microgrid project without fully diving into PyPSA EUR 

### Documentation for Use in PyPSA-EUR 

# SPDX-FileCopyrightText: Contributors to PyPSA-Eur <https://github.com/pypsa/pypsa-eur>
#
# SPDX-License-Identifier: MIT
"""
Calculates for each clustered region the (i) installable capacity (based on
land-use from :mod:`determine_availability_matrix`), (ii) the available
generation time series (based on weather data), and (iii) the average distance
from the node for onshore wind, AC-connected offshore wind, DC-connected
offshore wind and solar PV generators.

.. note:: Hydroelectric profiles are built in script :mod:`build_hydro_profiles`.

Outputs
-------

- ``resources/profile_{technology}.nc`` with the following structure

    ===================  ==========  =========================================================
    Field                Dimensions  Description
    ===================  ==========  =========================================================
    profile              bus, time   the per unit hourly availability factors for each bus
    -------------------  ----------  ---------------------------------------------------------
    p_nom_max            bus         maximal installable capacity at the bus (in MW)
    -------------------  ----------  ---------------------------------------------------------
    average_distance     bus         average distance of units in the region to the
                                     grid bus for onshore technologies and to the shoreline
                                     for offshore technologies (in km)
    ===================  ==========  =========================================================

    - **profile**

    .. image:: img/profile_ts.png
        :scale: 33 %
        :align: center

    - **p_nom_max**

    .. image:: img/p_nom_max_hist.png
        :scale: 33 %
        :align: center

    - **average_distance**

    .. image:: img/distance_hist.png
        :scale: 33 %
        :align: center

Description
-----------

This script functions at two main spatial resolutions: the resolution of the
clustered network regions, and the resolution of the cutout grid cells for the
weather data. Typically the weather data grid is finer than the network regions,
so we have to work out the distribution of generators across the grid cells
within each region. This is done by taking account of a combination of the
available land at each grid cell (computed in
:mod:`determine_availability_matrix`) and the capacity factor there.

Based on the availability matrix, the script first computes how much of the
technology can be installed at each cutout grid cell. To compute the layout of
generators in each clustered region, the installable potential in each grid cell
is multiplied with the capacity factor at each grid cell. This is done since we
assume more generators are installed at cells with a higher capacity factor.

.. image:: img/offwinddc-gridcell.png
    :scale: 50 %
    :align: center

.. image:: img/offwindac-gridcell.png
    :scale: 50 %
    :align: center

.. image:: img/onwind-gridcell.png
    :scale: 50 %
    :align: center

.. image:: img/solar-gridcell.png
    :scale: 50 %
    :align: center

This layout is then used to compute the generation availability time series from
the weather data cutout from ``atlite``.

The maximal installable potential for the node (`p_nom_max`) is computed by
adding up the installable potentials of the individual grid cells.
"""



In [ ]:
#connection timeout with this code, but the layout should be correct.
import time

import atlite
import geopandas as gpd
import xarray as xr
#from _helpers import configure_logging, get_snapshots, set_scenario_config
#from build_shapes import _simplify_polys
from dask.distributed import Client

#from build shapes
from itertools import takewhile
from operator import attrgetter

import country_converter as coco
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from rasterio.mask import mask
from shapely.geometry import MultiPolygon, Polygon, box

#logger = logging.getLogger(__name__)

#from other pypsa EUR files like _helpers
renewable_technologies = {
    "onwind": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "Vestas_V112_3MW",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 3,
        # "correction_factor": 0.93,
        "corine": {
            "grid_codes": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32],
            "distance": 1000,
            "distance_grid_codes": [1, 2, 3, 4, 5, 6]
        },
        "luisa": False,
        # "luisa": {
        #     "grid_codes": [1111, 1121, 1122, 1123, 1130, 1210, 1221, 1222, 1230, 1241, 1242],
        #     "distance": 1000,
        #     "distance_grid_codes": [1111, 1121, 1122, 1123, 1130, 1210, 1221, 1222, 1230, 1241, 1242]
        # },
        "natura": True,
        "excluder_resolution": 100,
        "clip_p_max_pu": 1e-2
    },
    "offwind-ac": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "NREL_ReferenceTurbine_2020ATB_5.5MW",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 2,
        "correction_factor": 0.8855,
        "corine": [44, 255],
        "luisa": False,  # [0, 5230]
        "natura": True,
        "ship_threshold": 400,
        "max_depth": 60,
        "max_shore_distance": 30000,
        "excluder_resolution": 200,
        "clip_p_max_pu": 1e-2,
        "landfall_length": 10
    },
    "offwind-dc": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "NREL_ReferenceTurbine_2020ATB_5.5MW",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 2,
        "correction_factor": 0.8855,
        "corine": [44, 255],
        "luisa": False,  # [0, 5230]
        "natura": True,
        "ship_threshold": 400,
        "max_depth": 60,
        "min_shore_distance": 30000,
        "excluder_resolution": 200,
        "clip_p_max_pu": 1e-2,
        "landfall_length": 10
    },
    "offwind-float": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "wind",
            "turbine": "NREL_ReferenceTurbine_5MW_offshore",
            "smooth": False,
            "add_cutout_windspeed": True
        },
        "capacity_per_sqkm": 2,
        "correction_factor": 0.8855,
        "corine": [44, 255],
        "natura": True,
        "ship_threshold": 400,
        "excluder_resolution": 200,
        "min_depth": 60,
        "max_depth": 1000,
        "clip_p_max_pu": 1e-2,
        "landfall_length": 10
    },
    "solar": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "pv",
            "panel": "CSi",
            "orientation": {
                "slope": 35.0,
                "azimuth": 180.0
            }
        },
        "capacity_per_sqkm": 5.1,
        # "correction_factor": 0.854337,
        "corine": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 26, 31, 32],
        "luisa": False,  # [1111, ..., 3330]
        "natura": True,
        "excluder_resolution": 100,
        "clip_p_max_pu": 1e-2
    },
    "solar-hsat": {
        "cutout": "europe-2013-sarah3-era5",
        "resource": {
            "method": "pv",
            "panel": "CSi",
            "orientation": {
                "slope": 35.0,
                "azimuth": 180.0,
                "tracking": "horizontal"
            }
        },
        "capacity_per_sqkm": 4.43,
        "corine": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 26, 31, 32],
        "luisa": False,  # [1111, ..., 3330]
        "natura": True,
        "excluder_resolution": 100,
        "clip_p_max_pu": 1e-2
    },
    "hydro": {
        "cutout": "europe-2013-sarah3-era5",
        "carriers": ["ror", "PHS", "hydro"],
        "PHS_max_hours": 6,
        "hydro_max_hours": "energy_capacity_totals_by_country",  # or estimate_by_large_installations, or float
        "flatten_dispatch": False,
        "flatten_dispatch_buffer": 0.2,
        "clip_min_inflow": 1.0,
        "eia_norm_year": False,
        "eia_correct_by_capacity": False,
        "eia_approximate_missing": False
    }
}
def get_snapshots(snapshots, drop_leap_day=False, freq="h", **kwargs):
    """
    Returns pandas DateTimeIndex potentially without leap days.
    """

    time = pd.date_range(freq=freq, **snapshots, **kwargs)
    if drop_leap_day and time.is_leap_year.any():
        time = time[~((time.month == 2) & (time.day == 29))]

    return time
def _simplify_polys(
    polys, minarea=100 * 1e6, maxdistance=None, tolerance=None, filterremote=True
):  # 100*1e6 = 100 km² if CRS is DISTANCE_CRS
    if isinstance(polys, MultiPolygon):
        polys = sorted(polys.geoms, key=attrgetter("area"), reverse=True)
        mainpoly = polys[0]
        mainlength = np.sqrt(mainpoly.area / (2.0 * np.pi))

        if maxdistance is not None:
            mainlength = maxdistance

        if mainpoly.area > minarea:
            polys = MultiPolygon(
                [
                    p
                    for p in takewhile(lambda p: p.area > minarea, polys)
                    if not filterremote or (mainpoly.distance(p) < mainlength)
                ]
            )
        else:
            polys = mainpoly
    if tolerance is not None:
        polys = polys.simplify(tolerance=tolerance)
    return polys

#options from configuration file 
technology_choice = 'offwind-dc'
snakemake_threads = 4 
snakemake_snapshots = ['']
snapshots = {
  'start': "2023-01-01",
  'end': "2024-01-01",
  'inclusive': 'left',
}
cutout_filepath = '/Users/katherine.shaw/Desktop/pypsa-eur/cutouts/europe-2023-sarah3-era5.nc'
availability_matrix_filepath_dc = '/Users/katherine.shaw/Desktop/pypsa-eur/resources/availability_matrix_39_offwind-dc.nc'
snakemake_input_regions = '/Users/katherine.shaw/Desktop/pypsa-eur/resources/regions_offshore_base_s_39.geojson'

nprocesses = snakemake_threads #int(snakemake.threads)
technology = technology_choice 
params = renewable_technologies[technology]
resource = params["resource"]  # pv panel params / wind turbine params

tech = next(t for t in ["panel", "turbine"] if t in resource)
models = resource[tech]
if not isinstance(models, dict):
    models = {0: models}
resource[tech] = models[next(iter(models))]

correction_factor = params.get("correction_factor", 1.0)
capacity_per_sqkm = params["capacity_per_sqkm"]

if correction_factor != 1.0:
    print(f"correction_factor is set as {correction_factor}")

if nprocesses > 1:
    client = Client(n_workers=nprocesses, threads_per_worker=1)
else:
    client = None

sns = get_snapshots(snapshots, drop_leap_day=False)

cutout = atlite.Cutout(cutout_filepath).sel(time=sns)

availability = xr.open_dataarray(availability_matrix_filepath_dc) #this is a file made my pypsa_eur 

regions = gpd.read_file(snakemake_input_regions) #this is also a file make by pypsa year
assert not regions.empty, (
    f"List of regions in {snakemake_input_regions} is empty, please "
    "disable the corresponding renewable technology"
)
# do not pull up, set_index does not work if geo dataframe is empty
regions = regions.set_index("name").rename_axis("bus")
if technology_choice.startswith("offwind"):
    # for offshore regions, the shortest distance to the shoreline is used
    offshore_regions = availability.coords["bus"].values
    regions = regions.loc[offshore_regions]
    regions = regions.map(lambda g: _simplify_polys(g, minarea=1)).set_crs(
        regions.crs
    )
else:
    # for onshore regions, the representative point of the region is used
    regions = regions.representative_point()
    regions = regions.geometry.to_crs(3035)
    buses = regions.index

    area = cutout.grid.to_crs(3035).area / 1e6
    area = xr.DataArray(
        area.values.reshape(cutout.shape), [cutout.coords["y"], cutout.coords["x"]]
    )

    func = getattr(cutout, resource.pop("method"))
    if client is not None:
        resource["dask_kwargs"] = {"scheduler": client}

    print(f"Calculate average capacity factor for technology {technology}...")
    start = time.time()

    capacity_factor = correction_factor * func(capacity_factor=True, **resource)
    layout = capacity_factor * area * capacity_per_sqkm

    duration = time.time() - start
    print(
        f"Completed average capacity factor calculation for technology {technology} ({duration:2.2f}s)"
    )

    profiles = []
    for year, model in models.items():
        print(
            f"Calculate weighted capacity factor time series for model {model} for technology {technology}..."
        )
        start = time.time()

        resource[tech] = model

        profile = func(
            matrix=availability.stack(spatial=["y", "x"]),
            layout=layout,
            index=buses,
            per_unit=True,
            return_capacity=False,
            **resource,
        )

        dim = {"year": [year]}
        profile = profile.expand_dims(dim)

        profiles.append(profile.rename("profile"))

        duration = time.time() - start
        print(
            f"Completed weighted capacity factor time series calculation for model {model} for technology {technology} ({duration:2.2f}s)"
        )

    profiles = xr.merge(profiles)

    print(f"Calculating maximal capacity per bus for technology {technology}")
    p_nom_max = capacity_per_sqkm * availability @ area

    print(f"Calculate average distances for technology {technology}.")
    layoutmatrix = (layout * availability).stack(spatial=["y", "x"])

    coords = cutout.grid.representative_point().to_crs(3035)

    average_distance = []
    for bus in buses:
        row = layoutmatrix.sel(bus=bus).data
        nz_b = row != 0
        row = row[nz_b]
        co = coords[nz_b]
        distances = co.distance(regions[bus]).div(1e3)  # km
        average_distance.append((distances * (row / row.sum())).sum())

    average_distance = xr.DataArray(average_distance, [buses])

    ds = xr.merge(
        [
            correction_factor * profiles,
            p_nom_max.rename("p_nom_max"),
            average_distance.rename("average_distance"),
        ]
    )
    # select only buses with some capacity and minimal capacity factor
    mean_profile = ds["profile"].mean("time")
    if "year" in ds.indexes:
        mean_profile = mean_profile.max("year")

    ds = ds.sel(
        bus=(
            (mean_profile > params.get("min_p_max_pu", 0.0))
            & (ds["p_nom_max"] > params.get("min_p_nom_max", 0.0))
        )
    )

    if "clip_p_max_pu" in params:
        min_p_max_pu = params["clip_p_max_pu"]
        ds["profile"] = ds["profile"].where(ds["profile"] >= min_p_max_pu, 0)

    
    if client is not None:
        client.shutdown()


In [ ]:
#wind and solar availability update 
##renewable profile for 2023
#the 2023 file from the respoitory above is for the year 2023
cutout_2023 = xr.load_dataset('~/pypsa-eur/cutouts/europe-2023-sarah3-era5.nc') 
#cutout defined from pypsa eur dataset
#cutout can be downloaded from this zenedo repository : https://zenodo.org/records/12791128


#2019 
#wind data 
wind_data = xr.load_dataset(
    "~/pypsa-eur/resources/profile_39_offwind-dc.nc",
)
wind_profile_df = wind_data["profile"].to_dataframe().reset_index()
wind_availability_list = wind_profile_df[wind_profile_df["bus"] == "GB2 0"]["profile"].to_numpy()

#solar data 
solar_profile = xr.load_dataset('~/pypsa-eur/resources/profile_39_solar.nc')
solar_availability_df = solar_profile['profile'].to_dataframe().reset_index()
solar_availability_df = solar_availability_df[solar_availability_df.bus == 'GB2 0']
solar_availability_df.reset_index()
solar_availability_df = solar_availability_df.profile
solar_availability = solar_availability_df.to_numpy()